# mongodb tutorial from 
https://github.com/dr-rodriguez/BlogTutorials/blob/master/notebooks/DataClass_MongoDB.ipynb

In [7]:
import json
from dataclasses import dataclass, field

@dataclass
class Coords:
    ra: float
    dec: float
        
@dataclass
class Photometry:
    value: float
    error: float
    unit: str = 'mag'
        
@dataclass
class BrownDwarf:
    source_id: int
    name: str
    coords: Coords
    J: Photometry = None
    H: Photometry = None
    Ks: Photometry = None
    spectral_type: str = None
    name_list: list = field(default_factory=list)
        
    def to_json(self):
        return json.dumps(self.__dict__, default=lambda x:x.__dict__, indent=4)

In [8]:
c = Coords(ra=181.889, dec=-39)
bd = BrownDwarf(source_id=11, name='1207-3832', coords=c,
               J = Photometry(12.99, 0.03),
               H = Photometry(12.39, 0.03),
               Ks = Photometry(11.95, 0.03))
bd.name_list = ['TWA 27', '2MASS J1207']
bd.spectral_type = 'M8.0'
print(bd)

BrownDwarf(source_id=11, name='1207-3832', coords=Coords(ra=181.889, dec=-39), J=Photometry(value=12.99, error=0.03, unit='mag'), H=Photometry(value=12.39, error=0.03, unit='mag'), Ks=Photometry(value=11.95, error=0.03, unit='mag'), spectral_type='M8.0', name_list=['TWA 27', '2MASS J1207'])


In [9]:
print(bd.to_json())

{
    "source_id": 11,
    "name": "1207-3832",
    "coords": {
        "ra": 181.889,
        "dec": -39
    },
    "J": {
        "value": 12.99,
        "error": 0.03,
        "unit": "mag"
    },
    "H": {
        "value": 12.39,
        "error": 0.03,
        "unit": "mag"
    },
    "Ks": {
        "value": 11.95,
        "error": 0.03,
        "unit": "mag"
    },
    "spectral_type": "M8.0",
    "name_list": [
        "TWA 27",
        "2MASS J1207"
    ]
}


In [10]:
import pymongo
import json

client = pymongo.MongoClient()    # default connection (ie, local)

db_name = 'test'
db = client[db_name]    # database
dwarfs = db.dwarfs
dwarfs.drop()    # drop collection, if needed

json_data = json.loads(bd.to_json())
result = dwarfs.insert_one(json_data)

# Quick check to confirm load
cursor = dwarfs.find({'source_id': 11})
for doc in cursor:
    print(doc)

{'_id': ObjectId('6007d279939a872ec4aaf181'), 'source_id': 11, 'name': '1207-3832', 'coords': {'ra': 181.889, 'dec': -39}, 'J': {'value': 12.99, 'error': 0.03, 'unit': 'mag'}, 'H': {'value': 12.39, 'error': 0.03, 'unit': 'mag'}, 'Ks': {'value': 11.95, 'error': 0.03, 'unit': 'mag'}, 'spectral_type': 'M8.0', 'name_list': ['TWA 27', '2MASS J1207']}


In [11]:
bddata = """#id	sname	ra	dec	sptype	J	J_err	H	H_err	names
2	1331-0116	202.95387	-1.280556	16	15.46	0.04	14.47	0.04	SDSS J133148.92-011651.4, 2MASS J13314894-0116500
4	1448+1031	222.106791	10.533056	13.5	14.556	0.034	13.433	0.033	2MASS J14482563+1031590, 2MASSW J1448256+103159, SDSS J144825.69+103158.8, 2MUCD 20587, ULAS2MASS J1448+1031, 2MASSW J1448256+103159
7	1439+1929	219.868167	19.487472	11	12.759	0.019	12.041	0.019	2MASS J14392837+1929150, 2MASSW J1439284+192915
14	2249+0044	342.472709	0.734611	11	16.587	0.125	15.421	0.109	SDSSp J224953.45+004404.2, 2M2249+0044
15	2208+2921	332.05679	29.355972	13	15.797	0.085	14.793	0.071	2MASSW J2208136+292121, 2MASS J22081363+2921215, 2MASSW J2208136+292121
17	0027+0503	6.924875	5.061583	8	16.189	0.093	15.288	0.099	2MASS J00274197+0503417, PC 0025+0447
19	2148+4003	327.068041	40.0665	16	14.147	0.029	12.783	0.03	2MASS J21481628+4003593, 2MASS J21481633+4003594, 2MASS J21481633+4003594, 2MASSW J2148162+400359, 2MASS J21481628+4003593
20	1102-3430	165.54097	-34.509869	8.5	13.034	0.024	12.356	0.022	TWA 28, 2MASS J11020983-3430355, SSSPM 1102-3431
32	0415-0935	63.831417	-9.585167	28	15.695	0.058	15.537	0.113	2MASSI J0415195-093506, 2MASS J04151954-0935066, WISEP J041521.20-093500.6
34	0727+1710	111.826001	17.167	27	15.6	0.061	15.756	0.171	2MASSI J0727182+171001, 2MASS J07271824+1710012, WISEP J072719.13+170952.0, LEHPM 2-461
36	0451-3402	72.753833	-34.0375	10.5	13.541	0.023	12.826	0.023	2MASSI J0451009-340214, V* SY Cae, 2MASS J04510093-3402150, 2MASSI J0451009-340214
53	1515+4847	228.753459	48.794889	16	14.111	0.029	13.099	0.031	2MASSW J1515008+484742, 2MASS J15150083+4847416, SDSS J151500.62+484744.8, 2MUCD 11314
61	1245-4429	191.309	-44.485477	9	14.518	0.032	13.8	0.033	TWA 29, 2MASS J12451416-4429077, DENIS J124514.1-442907
63	0334-4953	53.537667	-49.893944	9	11.376	0.023	10.823	0.026	2MASS J03341218-4953322, DENIS-P J033411.3-495333, 2MASSI J0334114-495334, LEHPM 3396
80	1552+2948	238.24591	29.81342	10	13.478	0.026	12.606	0.026	2MASSW J1552591+294849, 2MASS J15525906+2948485, 2MASSW J1552591+294849, SDSS J155259.01+294848.3
82	1835+3259	278.90792	32.998497	8.5	10.27	0.022	9.617	0.021	2MASS J18353790+3259545, LSR J1835+3259
83	1547-2423	236.94662	-24.397028	9	13.97	0.029	13.271	0.034	2MASS J15474719-2423493, DENIS J154747.2-242349
86	0036+1821	9.067376	18.352889	13.5	12.466	0.027	11.588	0.03	2MASSW J0036159+182110, 2MASS J00361617+1821104
91	0518-2756	79.692333	-27.946028	10	15.262	0.043	14.295	0.046	2MASS J05184616-2756457, 2MASS J05184616-2756457, 2MASSI J0518461-275645, 2MUCD 10381
96	0248-1651	42.170846	-16.856022	8	12.551	0.022	11.872	0.022	LP 771-21, 2MASS J02484096-1651249, BR 0246-1703, WISEP J024840.95-165124.8
98	0241-0326	40.297958	-3.449639	10	15.799	0.065	14.811	0.053	2MASS J02411151-0326587, 2MASS J02411151-0326587, 2MASSI J0241115-032658"""

In [12]:
data = []
for row in bddata.split('\n'):
    if row.startswith('#'): continue
    elems = row.split('\t')
    
    # Format the spectral type
    spnum = float(elems[4])
    if spnum >= 20:
        sptype = 'T{:3.1f}'.format(spnum-20)
    elif spnum >= 10:
        sptype = 'L{:3.1f}'.format(spnum-10)
    else:
        sptype = 'M{:3.1f}'.format(spnum)
        
    temp = BrownDwarf(source_id=int(elems[0]),
                      name=elems[1], 
                      coords=Coords(ra=float(elems[2]), dec=float(elems[3])),
                      spectral_type=sptype,
                      J=Photometry(value=float(elems[5]), error=float(elems[6])),
                      H=Photometry(value=float(elems[7]), error=float(elems[8])),
                      name_list=elems[9].split(',')
                     )
    data.append(temp)
            

In [13]:
for i, row in enumerate(data):
    if i>4: break  # only display the first 5
    print(row)

BrownDwarf(source_id=2, name='1331-0116', coords=Coords(ra=202.95387, dec=-1.280556), J=Photometry(value=15.46, error=0.04, unit='mag'), H=Photometry(value=14.47, error=0.04, unit='mag'), Ks=None, spectral_type='L6.0', name_list=['SDSS J133148.92-011651.4', ' 2MASS J13314894-0116500'])
BrownDwarf(source_id=4, name='1448+1031', coords=Coords(ra=222.106791, dec=10.533056), J=Photometry(value=14.556, error=0.034, unit='mag'), H=Photometry(value=13.433, error=0.033, unit='mag'), Ks=None, spectral_type='L3.5', name_list=['2MASS J14482563+1031590', ' 2MASSW J1448256+103159', ' SDSS J144825.69+103158.8', ' 2MUCD 20587', ' ULAS2MASS J1448+1031', ' 2MASSW J1448256+103159'])
BrownDwarf(source_id=7, name='1439+1929', coords=Coords(ra=219.868167, dec=19.487472), J=Photometry(value=12.759, error=0.019, unit='mag'), H=Photometry(value=12.041, error=0.019, unit='mag'), Ks=None, spectral_type='L1.0', name_list=['2MASS J14392837+1929150', ' 2MASSW J1439284+192915'])
BrownDwarf(source_id=14, name='2249+

In [14]:
for row in data:
    source_id = row.source_id
    json_data = json.loads(row.to_json())
    
    count = dwarfs.count_documents({'source_id': source_id})
    
    if count > 0:
        # Replace existing
        cursor = dwarfs.find({'source_id': source_id})
        for doc in cursor:
            result = dwarfs.replace_one({'_id': doc['_id']}, json_data)
    else:
        # Insert new
        result = dwarfs.insert_one(json_data)

In [15]:
count = dwarfs.count_documents({})
print('Total documents: ', count)
cursor = dwarfs.find({'spectral_type': 'M8.0'})
for doc in cursor:
    print(doc)

Total documents:  22
{'_id': ObjectId('6007d279939a872ec4aaf181'), 'source_id': 11, 'name': '1207-3832', 'coords': {'ra': 181.889, 'dec': -39}, 'J': {'value': 12.99, 'error': 0.03, 'unit': 'mag'}, 'H': {'value': 12.39, 'error': 0.03, 'unit': 'mag'}, 'Ks': {'value': 11.95, 'error': 0.03, 'unit': 'mag'}, 'spectral_type': 'M8.0', 'name_list': ['TWA 27', '2MASS J1207']}
{'_id': ObjectId('6007d917939a872ec4aaf187'), 'source_id': 17, 'name': '0027+0503', 'coords': {'ra': 6.924875, 'dec': 5.061583}, 'J': {'value': 16.189, 'error': 0.093, 'unit': 'mag'}, 'H': {'value': 15.288, 'error': 0.099, 'unit': 'mag'}, 'Ks': None, 'spectral_type': 'M8.0', 'name_list': ['2MASS J00274197+0503417', ' PC 0025+0447']}
{'_id': ObjectId('6007d917939a872ec4aaf195'), 'source_id': 96, 'name': '0248-1651', 'coords': {'ra': 42.170846, 'dec': -16.856022}, 'J': {'value': 12.551, 'error': 0.022, 'unit': 'mag'}, 'H': {'value': 11.872, 'error': 0.022, 'unit': 'mag'}, 'Ks': None, 'spectral_type': 'M8.0', 'name_list': ['LP

In [16]:
cursor = dwarfs.find({'spectral_type': 'M8.0'}, {'_id':0, 'source_id': 1, 'name': 1, 'spectral_type': 1})
for doc in cursor:
    print(doc)

{'source_id': 11, 'name': '1207-3832', 'spectral_type': 'M8.0'}
{'source_id': 17, 'name': '0027+0503', 'spectral_type': 'M8.0'}
{'source_id': 96, 'name': '0248-1651', 'spectral_type': 'M8.0'}


In [17]:
#dwarfs.drop_index('text_fields')
dwarfs.create_index([('name', pymongo.TEXT),
                    ('name_list', pymongo.TEXT)],
                   name = 'text_fields',
                   background=True)

'text_fields'

In [18]:
cursor = dwarfs.find({'$text': {'$search': 'TWA'}}, {'_id': 0, 'name': 1, 'name_list': 1})
for doc in cursor:
    print(doc)

{'name': '1245-4429', 'name_list': ['TWA 29', ' 2MASS J12451416-4429077', ' DENIS J124514.1-442907']}
{'name': '1102-3430', 'name_list': ['TWA 28', ' 2MASS J11020983-3430355', ' SSSPM 1102-3431']}
{'name': '1207-3832', 'name_list': ['TWA 27', '2MASS J1207']}


In [19]:
cursor = dwarfs.find({'name_list': {'$regex': '12'}}, {'_id': 0, 'name': 1, 'name_list': 1})
for doc in cursor:
    print(doc)

{'name': '1207-3832', 'name_list': ['TWA 27', '2MASS J1207']}
{'name': '2208+2921', 'name_list': ['2MASSW J2208136+292121', ' 2MASS J22081363+2921215', ' 2MASSW J2208136+292121']}
{'name': '0727+1710', 'name_list': ['2MASSI J0727182+171001', ' 2MASS J07271824+1710012', ' WISEP J072719.13+170952.0', ' LEHPM 2-461']}
{'name': '1245-4429', 'name_list': ['TWA 29', ' 2MASS J12451416-4429077', ' DENIS J124514.1-442907']}
{'name': '0334-4953', 'name_list': ['2MASS J03341218-4953322', ' DENIS-P J033411.3-495333', ' 2MASSI J0334114-495334', ' LEHPM 3396']}
{'name': '0248-1651', 'name_list': ['LP 771-21', ' 2MASS J02484096-1651249', ' BR 0246-1703', ' WISEP J024840.95-165124.8']}


In [20]:
cursor = dwarfs.find({'source_id': 11})
doc = list(cursor)[0]
del doc['_id']
print(doc)

{'source_id': 11, 'name': '1207-3832', 'coords': {'ra': 181.889, 'dec': -39}, 'J': {'value': 12.99, 'error': 0.03, 'unit': 'mag'}, 'H': {'value': 12.39, 'error': 0.03, 'unit': 'mag'}, 'Ks': {'value': 11.95, 'error': 0.03, 'unit': 'mag'}, 'spectral_type': 'M8.0', 'name_list': ['TWA 27', '2MASS J1207']}


In [21]:
j = Photometry(**doc['J'])
h = Photometry(**doc['H'])
ks = Photometry(**doc['Ks'])
c = Coords(**doc['coords'])
print(j)
print(c)

Photometry(value=12.99, error=0.03, unit='mag')
Coords(ra=181.889, dec=-39)


In [22]:
del doc['coords']
del doc['J']
del doc['H']
del doc['Ks']
new_bd = BrownDwarf(**doc, coords=c, J=j, H=h, Ks=ks)
print(new_bd)

BrownDwarf(source_id=11, name='1207-3832', coords=Coords(ra=181.889, dec=-39), J=Photometry(value=12.99, error=0.03, unit='mag'), H=Photometry(value=12.39, error=0.03, unit='mag'), Ks=Photometry(value=11.95, error=0.03, unit='mag'), spectral_type='M8.0', name_list=['TWA 27', '2MASS J1207'])


In [23]:
# Original, for comparison
print(bd)

BrownDwarf(source_id=11, name='1207-3832', coords=Coords(ra=181.889, dec=-39), J=Photometry(value=12.99, error=0.03, unit='mag'), H=Photometry(value=12.39, error=0.03, unit='mag'), Ks=Photometry(value=11.95, error=0.03, unit='mag'), spectral_type='M8.0', name_list=['TWA 27', '2MASS J1207'])


In [24]:
bd == new_bd

True